1)
Импортируйте библиотеку pandas как pd. Прочитайте датасет bookings.csv с разделителем ;. Ссылка на датасет находится в переменной path_to_file.

Сохраните прочитанный датасет в bookings.

Затем запишите первые 7 строк в переменную bookings_head.

Решение:

import pandas as pd

bookings = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-rudenko/shared/homeworks/python_ds_miniprojects/2/bookings.csv', sep = ';')

bookings.head(7)

2)
Посмотрите на размеры таблицы. Сколько всего в ней столбцов?

Решение:

bookings.shape

3)
Как вы уже могли заметить, при сборе датасета произошла небольшая ошибка, и названия колонок были записаны в неудобном для дальнейшей работы формате. Исправьте ситуацию, приведя названия столбцов к нижнему регистру и заменив пробелы на знак нижнего подчеркивания. 

Решение:

bookings.columns

bookings = bookings.rename (columns = {'Hotel':'hotel', 'Is Canceled':'is_canceled', 'Lead Time':'lead_time', 'arrival full date':'arrival_full_date',
       'Arrival Date Year':'arrival_date_year', 'Arrival Date Month':'arrival_date_month', 'Arrival Date Week Number':'arrival_date_week_number',
       'Arrival Date Day of Month':'arrival_date_day_of_month', 'Stays in Weekend nights':'stays_in_weekend_nights',
       'Stays in week nights':'stays_in_week_nights', 'stays total nights':'stays_total_nights', 'Adults':'adults', 'Children':'children',
       'Babies':'babies', 'Meal':'meal', 'Country':'country', 'Reserved Room Type':'reserved_room_type', 'Assigned room type':'assigned_room_type',
       'customer type':'customer_type', 'Reservation Status':'reservation_status', 'Reservation status_date':'reservation_status_date'})

4) Перейдем к исследованию данных! Пользователи из каких стран совершили наибольшее число успешных бронирований? Бронирование считается успешным, если в дальнейшем не было отменено (переменная is_canceled). В качестве ответа выберите страны, входящие в топ-5.



Решение:

bookings

bookings.query ('is_canceled == 0')\
    .value_counts('country')
    

5) На сколько ночей (stays_total_nights)  в среднем бронируют отели типа City Hotel? Resort Hotel? Запишите полученные значения в пропуски с точностью до 2 знаков после точки.

Может пригодиться: round

Решение:

bookings.groupby(['hotel'], as_index = False)\
    .agg({'stays_total_nights':'mean'})\
    .round(2)

6)Иногда тип номера, присвоенного клиенту (assigned_room_type), отличается от изначально забронированного (reserved_room_type).
Такое может произойти, например, по причине овербукинга.
Сколько подобных наблюдений встретилось в датасете?
*отмена бронирования также считается

bookings.query('assigned_room_type != reserved_room_type').value_counts('hotel').sum() 

6)Теперь проанализируйте даты запланированного прибытия (arrival_date_year).

На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017?

Решение:

bookings.query('arrival_date_year ==2017').value_counts('arrival_date_month')

7)Сгруппируйте данные по годам, а затем проверьте, на какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 

Hint: для этого сначала примените метод groupby, после чего выберите нужную колонку для дальнейших действий :)

Первое решение:

bookings.query ("hotel == 'City Hotel' and is_canceled ==1")\
    .groupby ('arrival_date_year')['arrival_date_month']\
    .value_counts ('arrival_date_month')

Второе решение:
В запросе поменять в переменной arrival_date_year на нужный год

bookings.query("hotel =='City Hotel' and is_canceled ==1 and arrival_date_year == 2017")\
    .value_counts('arrival_date_month')

8) Посмотрите на числовые характеристики трёх колонок: adults, children и babies. Какая из них имеет наибольшее среднее значение?

Решение:

bookings[['adults','children','babies']].mean()

9) Создайте колонку total_kids, объединив столбцы children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим?

City hotel – отель находится в городе
Resort hotel – отель курортный
В качестве ответа укажите наибольшее среднее total_kids, округлив до 2 знаков после точки.

Решение:

bookings['total_kids'] = bookings['children'] + bookings['babies']
bookings.groupby('hotel')\
    .agg({'total_kids':'mean'})\
    .sort_values('hotel')\
    .round(2)\
    .max()

10) Не все бронирования завершились успешно (is_canceled), поэтому можно посчитать, сколько клиентов было потеряно в процессе. Иными словами, посчитать метрику под названием Churn Rate.

Churn rate (отток, коэффициент оттока) – это процент подписчиков (например, на push-уведомления от сайта), которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Иными словами, представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах.

В нашем случае Churn Rate - это процент клиентов, которые отменили бронирование. Давайте посмотрим, как эта метрика связана с наличием детей у клиентов!

Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Далее проверьте, среди какой группы пользователей показатель оттока выше. 

В качестве ответа укажите наибольший % оттока, округленный до 2 знаков после точки (то есть доля 0.24563 будет 24.56% и в ответ пойдёт 24.56)

Решение:

In [277]:
bookings['has_kids'] = bookings.total_kids != 0


In [386]:
a = bookings.query('has_kids == True and is_canceled == 1').value_counts('is_canceled') # пары с детьми, которые отменили бронь

In [402]:
b = bookings.query('has_kids == True').value_counts('is_canceled').sum() #пары с детьми

In [391]:
c = bookings.query('has_kids == False and is_canceled == 1').value_counts('is_canceled') #пары без детей, отменившие бронь


In [400]:
d = bookings.query('has_kids == False').value_counts('is_canceled').sum() #пары без детей

In [409]:
canc_kids = ((bookings.query('has_kids == True and is_canceled == 1').value_counts('is_canceled') / b) * 100).max().round (2)

In [415]:
canc_no_kids = ((c / d) * 100).max().round (2)

In [417]:
if canc_kids > canc_no_kids:
    print (canc_kids)
else:
    print (canc_no_kids)

37.22
